In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.5.0
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"
!pip install ninja packaging
!MAX_JOBS=4 pip install flash-attn --no-build-isolation


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.2.1-cp310-cp310-linux_x86_64.whl size=193960420 sha256=90f73bf1b0351f988d545836abd0f27aca8f72040720003809003b0ed40685a0
  Stored in directory: /root/.cache/pip/wheels/29/3d/d0/ca80c9c6061afa997cd0246a982fd905ef3a1837c8442a0050
Successfully built flash-attn


In [3]:
from typing import Dict, List, Union

In [4]:
def return_prompt_and_responses(samples) -> Dict[str, Union[str, List[str]]]:
    return {
        "prompt": [
            "Question: " + question + "\n\nAnswer: "
            for question in samples["question"]
        ],
        "chosen": samples["response_j"],   # rated better than k
        "rejected": samples["response_k"], # rated worse than j
    }


In [5]:
from datasets import load_dataset

In [6]:
dataset = load_dataset(
    "lvwerra/stack-exchange-paired",
    split="train",
    data_dir="data/rl"
)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
    num_rows: 7435908
})

In [8]:
print(dataset[0])


{'qid': 53179085, 'question': 'I\'m writing a code using Java Swing to press the right button when I type a number key.\nBut I can\'t find what I want through search.\nThis is my code and I can\'t understand why this isn\'t working.\nPlease help me..\n\n```\nimport javax.swing.*;\nimport java.awt.Dimension;\nimport java.awt.event.*;\n\nclass class01 {\n\n    public static void main(String[] args) {\n\n        JFrame f = new JFrame("Key event test"); \n        f.setSize(230, 500);\n        f.setLayout(null);\n        f.setVisible(true);\n        f.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);\n\n        JLabel label = new JLabel(); \n\n        JButton button1 = new JButton("Coffe"); \n        button1.setSize(100, 100);\n        button1.setLocation(0, 0);\n\n        JButton button2 = new JButton("Latte");\n        button2.setSize(100, 100);\n        button2.setLocation(0, 100);\n\n        JButton button3 = new JButton("Espresso");\n        button3.setSize(100, 100);\n        button3.se

In [9]:
original_columns = dataset.column_names

In [10]:
dataset = dataset.shuffle(seed=42).select(range(2000))

In [11]:
dataset

Dataset({
    features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
    num_rows: 2000
})

In [12]:
dataset.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2000
})

In [13]:
dataset

Dataset({
    features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
    num_rows: 2000
})

In [14]:
dataset = dataset.train_test_split(seed=42, shuffle=True, test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 1800
    })
    test: Dataset({
        features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
        num_rows: 200
    })
})

In [15]:
dataset["train"][0]

{'qid': 2632256,
 'question': "I'm trying to integrate the following: \n\n$$ 2\\lambda\\cdot \\int\\_0^\\infty x^2e^{-2\\lambda x} $$\n\nThe answer is supposed to be $\\frac 1{4\\lambda^2}$ however I'm attempting to solve it via a simple double application of integration by parts and keep getting $2$. \n\nI'm unsure how to solve this otherwise, but mostly I'm not sure why a double application of integration by parts doesn't produce the solution. I've run through my work a few times now and can't find a mistake so I feel like there must be a greater principle I'm missing. Why can't I just integrate by parts twice to nuke the first term? \n\nMy attempt at solution:\n\nTaking the integral of the first expression, I apply integration by parts for:\n\n$$ \\frac {-x^2\\*e^{-2\\lambda x}}{2\\lambda} - 2\\int xe^{-2\\lambda x}$$\n\nThis done, I do the same to the second term, receiving:\n\n$$ \\frac {-x\\*e^{-2\\lambda x}}{\\lambda} - 2\\int {e^{-2\\lambda x}} $$\n\nEvaluating the last integra

In [16]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

In [17]:
model_id = "NousResearch/Llama-2-7b-hf"

In [18]:
use_flash_attention = True

# replace attention with flash attention
if torch.cuda.get_device_capability()[0] >= 8:
  from utils.llama_patch import replace_attn_with_flash_attn
  print("Using flash attention")
  replace_attn_with_flash_attn()
  use_flash_attention = True


Using flash attention


In [19]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [21]:
# load the base model in 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
    use_auth_token=True,
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1


# Validate that the model is using flash attention, by comparing doc strings
if use_flash_attention:
    from utils.llama_patch import forward
    assert base_model.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"



tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# add LoRA layers on top of the quantized base model
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
base_model = prepare_model_for_kbit_training(base_model)
base_model = get_peft_model(base_model, peft_config)

In [23]:
from utils.llama_patch import upcast_layer_for_flash_attention
model = upcast_layer_for_flash_attention(base_model, torch.bfloat16)

In [24]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="llama-7-int4-stack-exchange",
    max_steps=100,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="steps",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)


In [25]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=return_prompt_and_responses,
    args=training_args,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:227: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:469: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


In [ ]:
# save model
trainer.save_model()

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
    script_args.model_name_or_path, # location of saved SFT model
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    is_trainable=True,
)
model_ref = AutoPeftModelForCausalLM.from_pretrained(
    script_args.model_name_or_path,  # same model as the main one
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
...
dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=0.1,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    peft_config=peft_config,
)
dpo_trainer.train()
dpo_trainer.save_model()
